# Copy Kernel Test
Este cuaderno prueba el kernel `copy_kernel` copiando una matriz de números complejos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pynq import Overlay, allocate
%matplotlib inline

In [ ]:
ol = Overlay('sistema.bit')  # Reemplace con el bitstream que contiene copy_kernel
ip = ol.copy_kernel_0

In [ ]:
N = 64
TOTAL = N * N
x = np.linspace(-3, 3, N, dtype=np.float32)
X, Y = np.meshgrid(x, x)
sigma = 0.8
gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2)).astype(np.float32)
in_matrix = gaussian.astype(np.complex64)
in_buffer = allocate(shape=(TOTAL,), dtype=np.complex64)
out_buffer = allocate(shape=(TOTAL,), dtype=np.complex64)
np.copyto(in_buffer, in_matrix.reshape(TOTAL))

In [ ]:
ip.write(0x10, in_buffer.physical_address)
ip.write(0x18, out_buffer.physical_address)
ip.write(0x00, 1)  # ap_start
while (ip.read(0x00) & 0x2) == 0:
    pass
out_matrix = np.array(out_buffer).reshape(N, N)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].imshow(np.abs(in_matrix))
axes[0].set_title('Entrada |in|')
axes[1].imshow(np.abs(out_matrix))
axes[1].set_title('Salida |out|')
diff = out_matrix - in_matrix
axes[2].imshow(np.abs(diff))
axes[2].set_title('|in - out|')
for ax in axes: ax.axis('off')
plt.show()
print('Error máximo:', np.max(np.abs(diff)))